# Week3-InteractiveVisualization
This notebook loads a multi-site dataset of subcortical nucleus (thalamus, globus pallidus, and striatum) volumes, performs site harmonization on these volumes using ComBat, then displays a number of interactive figures to explore the harmonized data.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import neuroCombat as nc

## Load and Prepare Data

In [3]:
data = pd.read_csv("data_trimmed.csv", index_col=0)

In [4]:
# Map site IDs to integers, as required by neuroCombat()
sites = data['Site'].unique()
site_dict = dict(zip(sites, range(1, len(sites)+1)))
data['Site_no'] = data['Site'].map(site_dict)
#data['Site'].map(site_dict)

## ComBat Site Harmonization

In [5]:
# The list of feature names
features = ['L_str_vol', 'L_GP_vol', 'L_thal_vol', 'R_str_vol', 'R_GP_vol', 'R_thal_vol']

# Perform harmonization with ComBat
harmonized_features = nc.neuroCombat(data[features].transpose(), covars=data[['Site_no', 'Age', 'DX']], batch_col='Site_no', categorical_cols=[], continuous_cols=[]).transpose()
harmonized_features = pd.DataFrame(harmonized_features)
harmonized_features.columns = features

# Add back site data and measures that were not harmonized.
harmonized_features['Site'] = data['Site']
harmonized_features['Age'] = data['Age']
harmonized_features['TBV'] = data['TBV']
harmonized_features['DX'] = data['DX']       # I sure hope that ComBat doesn't change the order of the rows!!!


[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data


## Compute linear regression models on the harmonized data.
Regress nucleus volume against diagnosis and age.

In [6]:
# Linear regression on ComBat harmonized data
from statsmodels.formula.api import ols

models = pd.Series(dtype='object')
model_params = pd.Series(dtype = 'object')
for nucleus in harmonized_features.columns[:-4]:        # Don't include last (covariate) columns
    models[nucleus] = ols(formula = nucleus + ' ~ DX + Age', data = harmonized_features).fit()
    print(models[nucleus].summary())
    model_params[nucleus] = models[nucleus].params
    print(model_params[nucleus])

                            OLS Regression Results                            
Dep. Variable:              L_str_vol   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.1438
Date:                Thu, 11 Jun 2020   Prob (F-statistic):              0.866
Time:                        12:43:49   Log-Likelihood:                -3024.7
No. Observations:                 359   AIC:                             6055.
Df Residuals:                     356   BIC:                             6067.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       1.05e+04    161.392     65.053

## Compute Linear Regression on Unharmonized Data

In [7]:
# Linear regression on unharmonized data
models_unh = pd.Series(dtype='object')
model_unh_params = pd.Series(dtype = 'object')
for nucleus in features:
    models_unh[nucleus] = ols(formula = nucleus + ' ~ DX + Age', data = data).fit()
    print(models_unh[nucleus].summary())
    model_unh_params[nucleus] = models_unh[nucleus].params
    print(model_unh_params[nucleus])

                            OLS Regression Results                            
Dep. Variable:              L_str_vol   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.4976
Date:                Thu, 11 Jun 2020   Prob (F-statistic):              0.608
Time:                        12:43:49   Log-Likelihood:                -3034.0
No. Observations:                 359   AIC:                             6074.
Df Residuals:                     356   BIC:                             6086.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       1.06e+04    165.622     64.016

## Interactive Visualizations


In [8]:
# Set up ipywidgets for interactive chart.
!jupyter nbextension enable --py widgetsnbextension
import ipywidgets as widgets
from ipywidgets import interact, fixed

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


### Scatterplots of nucleus volume against Age and Total Brain Volume
### Split violin plot showing original and harmonized distributions for each site

In [9]:
# Function to plot original and harmonized distributions together, by site.
def plot_dual_distributions(x, y, data1, data2, scale = 'count', ax=None):
    # Plots two distibutions (e.g. harmonized and unharmonized data) as the two halves of stacked violin plots.
    plt.figure()
    vp_data = pd.DataFrame({x: data1[x], y: data1[y], 'Legend': 'Unharmonized'})
    vp_data = vp_data.append(pd.DataFrame({x: data2[x], y: data2[y], 'Legend': 'Harmonized'}))
    ax_h_uh = sns.violinplot(y = y, x = x, data = vp_data, hue = 'Legend', split = True, scale = scale, ax = ax)
    #ax_h_uh.legend().remove()
    plt.show()

# Plot a panel of figures useful for quick data exploration and verification.
def plot_panel(nucleus, data1, data2):
    fig, axes = plt.subplots(ncols=2)
    plt.subplots_adjust(wspace = 0.5)
    sns.scatterplot('Age', nucleus, data = data, hue = "DX", ax=axes[0])
    sns.scatterplot('TBV', nucleus, data = data, hue = "DX", ax=axes[1])
    plot_dual_distributions(x=nucleus, y = 'Site', data1 = data1, data2 = data2, scale = 'count')

interact(plot_panel, nucleus = features, data1 = fixed(data), data2=fixed(harmonized_features))

interactive(children=(Dropdown(description='nucleus', options=('L_str_vol', 'L_GP_vol', 'L_thal_vol', 'R_str_v…

<function __main__.plot_panel(nucleus, data1, data2)>

### Plot fitted values against residuals for the ComBat-harmonized data

In [13]:
# Function to plot fitted values against residuals
def plot_fitted_vs_residual(models, ex_data, x_fitted_col, y_resid_col, display_col, hue_col, title_add = None):
    plt.figure()
    fitted_y = models[display_col].fittedvalues
    resid = models[display_col].resid
    # The following is not ideal. Need to revisit this solution.
    fit_resid_df = pd.DataFrame({'Residual': resid, 'Fitted_Value': fitted_y, 'Site': ex_data['Site'], 'DX': ex_data['DX']})
    ax = sns.scatterplot(fitted_y, resid, hue = hue_col, data=fit_resid_df)
    ax.set_title('Fitted vs. Residual Plot for ' + display_col + title_add)
    ax.set(xlabel = 'Fitted', ylabel = 'Residual')
    ax.legend().remove()
    plt.show()    

In [18]:
# Plot fitted values vs. residuals of ComBat harmonized data

interact(plot_fitted_vs_residual, 
         models = fixed(models), 
         ex_data = fixed(harmonized_features),
         x_fitted_col = fixed('Fitted_Value'), 
         y_resid_col = fixed('Residual'), 
         display_col = features, 
         hue_col = ['DX', 'Site'],
         title_add = fixed(' ComBat Harmonized'))

interactive(children=(Dropdown(description='display_col', options=('L_str_vol', 'L_GP_vol', 'L_thal_vol', 'R_s…

<function __main__.plot_fitted_vs_residual(models, ex_data, x_fitted_col, y_resid_col, display_col, hue_col, title_add=None)>

In [19]:
# Plot fitted values vs. residuals of unharmonized data

interact(plot_fitted_vs_residual, 
         models = fixed(models_unh), 
         ex_data = fixed(data),
         x_fitted_col = fixed('Fitted_Value'), 
         y_resid_col = fixed('Residual'), 
         display_col = features, 
         hue_col = ['DX', 'Site'],
         title_add = fixed(' Unharmonized'))

interactive(children=(Dropdown(description='display_col', options=('L_str_vol', 'L_GP_vol', 'L_thal_vol', 'R_s…

<function __main__.plot_fitted_vs_residual(models, ex_data, x_fitted_col, y_resid_col, display_col, hue_col, title_add=None)>